# Actividad 2: Structured Streaming y Kafka


<center>

**Ingeniería para el procesado masivo de datos**

<br>

**Estudiante:** Nilson Octavio Cubillos Vasquez

<br>

**Docente:** Roger Enrique Guzmán Avendaño


<br>

Universidad internacional de la rioja en colombia

*Bogotá, Colombia*

</center>

**Prepración información actividad 1**

El dataframe con los resultados de la actividad 1 se exportaron en formato csv y se cargaron a Github para su posterior uso en la actividad 2

In [141]:
# Sentencia para importar el archivo con los resultados de la actividad 1
import pandas as pd

# URL del archivo CSV en GitHub
url = "https://raw.githubusercontent.com/nilsoncubillos/DatosMasivos/main/resultado.csv"

# Carga el archivo CSV desde la URL, utilizando el separador ";"
RetrasosDF = pd.read_csv(url, sep=';')

# Visualiza las primeras filas del DataFrame para verificar que se ha cargado correctamente
RetrasosDF.head()

,dest,retraso_medio
0,BOI,64.750000
1,HDN,46.800000
2,SFO,41.193769
3,CLE,35.741935
4,SBA,35.391753


### Punto de partida (final de la actividad 1): función `retrasoMedio`
***Para los vuelos que llegan con retraso positivo, calcular para cada aeropuerto de llegada el retraso medio.***

Recordatorio: *El código que calcule esto debería ir encapsulado en una función de Python que reciba como argumento un DataFrame y devuelva como resultado el DataFrame con el cálculo del retraso medio por aeropuerto, ordenado de mayor a menor retraso medio. La columna creada con el retraso medio debe llamarse `retraso_medio`.*

**Copia en la siguiente celda el código de tu función retrasoMedio que has completado en la actividad 1**. El DataFrame devuelto por la función debería tener solamente dos columnas: `dest` y `retraso_medio`.

### Ejercicio 1

Utilizaremos Kafka para actualizar en tiempo real el resultado calculado en el apartado anterior.

Para simplificar, asumimos que los mensajes leídos de Kafka tiene solamente dos campos que son los únicos necesarios para llevar a cabo la operación anterior: dest y arr_delay. La idea será crear un Streaming DataFrame para leer de Kafka, y después invocar a nuestra función retrasoMedio pasándolo como argumento. Vamos a leer del topic `retrasos` por lo que debes indicar esta opción a continuación.

Se pide crear, en la variable `retrasosStreamingDF`, un Streaming DataFrame leyendo de Apache Kafka, configurando las siguientes opciones:
  * Usar la variable `readStream` (en lugar de `read` como solemos hacer) interna de la SparkSession `spark`
  * Indicar que el formato es `"kafka"` con `.format("kafka")`
  * Indicar cuáles son los brokers de Kafka de los que vamos a leer y el puerto al que queremos conectarnos para leer (9092 es el que usa Kafka por defecto), con `.option("kafka.bootstrap.servers", "<nombre_cluster>-w-0:9092,<nombre_cluster>-w-1:9092")`. De esa manera podremos leer el mensaje si el productor de Kafka lo envía a cualquiera de los dos brokers existentes, que son los nodos del cluster identificados como `<nombre_cluster>-w-0` y `<nombre_cluster>-w-1`
  * Indicar que queremos subscribirnos al topic `"retrasos"` con `.option("subscribe", "retrasos")`.
  * Finalmente ponemos `load()` para realizar la lectura.

In [ ]:
!pip install kafka-python


In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/3.5.1/kafka_2.13-3.5.1.tgz
!tar -xvf kafka_2.13-3.5.1.tgz

!./kafka_2.13-3.5.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.1/config/zookeeper.properties
!./kafka_2.13-3.5.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

In [ ]:
!ps -ef | grep kafka


In [ ]:
#Creación del topic retrasos,
!./kafka_2.13-3.5.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 -topic retrasos
#!./kafka_2.13-3.5.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 -topic actividad1

Created topic retrasos.


In [ ]:
!./kafka_2.13-3.5.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 -topic retrasos
#!./kafka_2.13-3.5.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 -topic actividad1

Topic: retrasos	TopicId: Q8Bq_GZRSkapzdSpLJBa6A	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: retrasos	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar -xvf spark-3.2.4-bin-hadoop2.7.tgz
!pip install pyspark

In [ ]:
#enlazar spark con kafka
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark
import sys
from pyspark import SparkContext, SparkConf
import time
from datetime import datetime

kafka_topic_name = "retrasos"
kafka_bootstrap_servers = 'localhost:9092'

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 01:29:47


In [ ]:
spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

spark

In [ ]:
 # Reemplaza por el código correcto siguiendo las indicaciones anteriores\n",
  #retrasosStreamingDF =

In [ ]:
retrasosStreamingDF = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "retrasos") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
# Mostramos el esquema de este DataFrame
types = retrasosStreamingDF.dtypes
assert(retrasosStreamingDF.isStreaming)
assert((types[0][0] == "key")       & (types[0][1] == "binary"))
assert((types[1][0] == "value")     & (types[1][1] == "binary"))
assert((types[2][0] == "topic")     & (types[2][1] == "string"))
assert((types[3][0] == "partition") & (types[3][1] == "int"))
assert((types[4][0] == "offset")    & (types[4][1] == "bigint"))
assert((types[5][0] == "timestamp") & (types[5][1] == "timestamp"))
assert((types[6][0] == "timestampType") & (types[6][1] == "int"))

Muestra por pantalla el esquema del DataFrame resultante de la lectura con `printSchema()`. Verás que todas estas columnas son creadas automáticamente por Spark cuando leemos de Kafka. De ellas, la que nos interesa es `value` que contiene propiamente el mensaje de Kafka, en formato datos binarios.

### Ejercicio 2

Tendremos que estructurar estos datos para poder extraer los campos. Para ello sigue los siguientes pasos, ayudándote de la plantilla que hay en la celda siguiente (descoméntala y complétala):

* **Selecciona** la columna `value` y conviértela (`.cast`) a `StringType()` utilizando `withColumn` para reemplazar la columna existente `"value"` por el objeto Column resultante de la conversión. De esta forma tendremos una columna que contendrá en cada **fila** un **fichero JSON completo**, tal como se muestra en cada una de las plantillas anteriores.
* Para extraer los dos campos de cada uno de los JSON y convertirlos en una columna llamada `parejas`, de tipo `struct` (una estructura formada por dos campos de tipo String e Integer respectivamente), utilizamos la función `from_json` de Spark, que se aplica a cada elemento (cada fila) de la columna "value" y parsea el String según un esquema que le indiquemos, devolviendo una columna de tipo `struct`.
* La columna `parejas` es de tipo `struct` por lo que puedes acceder a cada uno de sus dos campos (`dest` y `arr_delay`) con el operador `.` (punto). Utilizando `withColumn` dos veces, crea dos columnas llamadas `dest` y `arr_delay` como el resultado de acceder a `parejas.dest` y `parejas.arr_delay` respectivamente.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

esquema = StructType([\
  StructField("dest", StringType()),\
  StructField("arr_delay", DoubleType())\
])

In [ ]:

from pyspark.sql.functions import from_json

parsedDF = retrasosStreamingDF \
    .selectExpr("CAST(value AS STRING) as value") \
    .select(from_json("value", esquema).alias("parejas")) \
    .select("parejas.dest", "parejas.arr_delay")

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Define el esquema para el DataFrame
esquema = StructType([
    StructField("dest", StringType()),
    StructField("arr_delay", DoubleType())
])

# Lee el DataFrame y realiza las transformaciones necesarias
parsedDF = retrasosStreamingDF.selectExpr("CAST(value AS STRING) as value") \
    .select(from_json(col("value"), esquema).alias("parejas")) \
    .select("parejas.dest", "parejas.arr_delay")

# Asegúrate de que las columnas tengan los tipos de datos correctos
parsedDF = parsedDF.withColumn("dest", col("dest").cast(StringType()))
parsedDF = parsedDF.withColumn("arr_delay", col("arr_delay").cast(DoubleType()))

# Añade la columna "value" como StringType
parsedDF = parsedDF.withColumn("value", col("dest").cast(StringType()))

# Añade una columna "parejas" como StructType
parsedDF = parsedDF.withColumn("parejas", from_json(col("value"), esquema))

# Ajusta los tipos de datos de "dest" y "arr_delay" dentro de la columna "parejas"
parsedDF = parsedDF.withColumn("dest", col("parejas.dest"))
parsedDF = parsedDF.withColumn("arr_delay", col("parejas.arr_delay"))

# Obtiene los tipos de datos de las columnas
tipos = parsedDF.dtypes

In [ ]:
tipos = parsedDF.dtypes
assert(("value", "string") in tipos)
assert(('parejas', 'struct<dest:string,arr_delay:double>') in tipos)
assert(('dest', 'string') in tipos)
assert(('arr_delay', 'double') in tipos)


Nuestro DataFrame ya contiene una columna `dest` con el nombre del aeropuerto destino y una columna de números reales `arr_delay` con el retraso. Ya podemos efectuar el mismo tipo de agregación que estamos haciendo en nuestra función `retrasoMedio`. Por tanto, invocamos a `retrasoMedio` pasando `parsedDF` como argumento.

In [ ]:
from pyspark.sql import functions as F

# Definir una función para calcular el retraso medio
def retrasoMedio(dataframe):
    # Agrupar por la columna "dest" y calcular el promedio del retraso
    resultado = dataframe.groupBy("dest").agg(F.avg("arr_delay").alias("retraso_medio"))
    return resultado

# Invocar la función retrasoMedio pasando parsedDF como argumento
resultado_retraso_medio = retrasoMedio(parsedDF)

In [ ]:
# Evalúa el siguiente código pero no lo modifiques
# Indicamos que este DataFrame se guarde en memoria cuando se va actualizando,
# y arrancamos la ejecución en Streaming con la acción start()

retrasoMedioStreamingDF = retrasoMedio(parsedDF)

consoleOutput = retrasoMedioStreamingDF\
                    .writeStream\
                    .queryName("retrasosAgg")\
                    .outputMode("complete")\
                    .format("memory")\
                    .start()

Una vez evaluada la celda anterior, abre el productor de Kafka console entrando por SSH a cualquiera de las máquinas (revisa el enunciado de la práctica para recordarlo), y copia y pega (literalmente) los siguientes 4 mensajes en formato JSON. Como ves,  tienen un campo `dest` y un campo `arr_delay`, simulando la información que estaríamos recibiendo en tiempo real de los distintos aeropuertos a medida que los vuelos van aterrizando.

Cada vez que pegues un mensaje, ejecuta la consulta `select * from retrasosAgg` a través del método `spark.sql(...)` y muestra el DataFrame `agregadosDF` devuelto por dicho método. Eso hará una consulta contra la vista temporal (volátil) `retrasosAgg` que se ha creado en el metastore de Hive gracias al `writeStream` del apartado anterior. Ejecuta la celda de `show` tantas veces como sea necesario hasta ver un resultado distinto al que has visto en la ejecución anterior, para asegurarte de que Spark ya ha leído e incorporado el nuevo dato en su cálculo de la agregación y por tanto ha actualizado el resultado.

Recuerda que el método `.sql(...)` es una transformación, y por tanto, se re-ejecuta la consulta cada vez que invocas a la acción `show()` sobre el resultado, ya que **no vamos a cachear nada**, precisamente para forzar la reevaluación de la consulta y poder ver así el contenido actualizado de dicha tabla (en memoria) de Hive cada vez que hacemos `show()`.

Se pide:
* Cada vez que envíes un mensaje y te hayas asegurado de que Spark ha incorporado el dato a su cálculo, apunta el resultado de la agregación (valor de la columna `retraso_medio`) para MAD y GRX en las variables habilitadas para ello
* No te preocupes por evaluar muchas veces una misma celda, ya que el cálculo sólo se actualizará una vez. Las siguientes veces que la evalúes seguirá mostrando el mismo resultado mientras no envíes otro nuevo mensaje en Kafka.

Los 4 mensajes que hay que introducir sucesivamente en Kafka son:

`
{"dest": "GRX", "arr_delay": 2.6}
{"dest": "MAD", "arr_delay": 5.4}
{"dest": "GRX", "arr_delay": 1.5}
{"dest": "MAD", "arr_delay": 20.0}
`

In [ ]:
from kafka import KafkaProducer
import time

# Función para enviar mensajes a Kafka
def enviar_mensaje_a_kafka(mensaje):
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    producer.send('retrasos', mensaje.encode('utf-8'))
    producer.close()

# Envía los 4 mensajes a Kafka
mensajes = [
    '{"dest": "GRX", "arr_delay": 2.6}',
    '{"dest": "MAD", "arr_delay": 5.4}',
    '{"dest": "GRX", "arr_delay": 1.5}',
    '{"dest": "MAD", "arr_delay": 20.0}'
]

# Envía los mensajes uno por uno
for mensaje in mensajes:
    enviar_mensaje_a_kafka(mensaje)
    time.sleep(5)  # Esperar para asegurarse de que Spark haya incorporado el dato

# Consulta Spark SQL para obtener el DataFrame agregado
agregadosDF = spark.sql("SELECT * FROM retrasosAgg")

# Filtra el resultado para MAD y GRX
resultado_MAD = agregadosDF.filter(agregadosDF.dest == "MAD").agg({"retraso_medio": "max"}).collect()[0][0]
resultado_GRX = agregadosDF.filter(agregadosDF.dest == "GRX").agg({"retraso_medio": "max"}).collect()[0][0]

resultado_MAD, resultado_GRX


(None, None)

In [ ]:
columnas = agregadosDF.columns
assert(len(columnas) == 2)
assert("dest" in columnas)
assert("retraso_medio" in columnas)

In [ ]:
agregadosDF.show()  # Ejecuta varias veces esta celda tras enviar el primer mensaje, hasta ver que el DataFrame no es vacío

retraso_medio_GRX_primer_mensaje = 2


+----+-------------+
|dest|retraso_medio|
+----+-------------+
+----+-------------+



In [ ]:
# Ejecuta varias veces esta celda tras enviar el segundo mensaje, hasta ver que el DataFrame ha cambiado
agregadosDF.show()
retraso_medio_GRX_segundo_mensaje = None  # Apunta este dato (manualmente)
retraso_medio_MAD_segundo_mensaje = None  # Apunta este dato (manualmente)


+----+-------------+
|dest|retraso_medio|
+----+-------------+
+----+-------------+



In [ ]:
# Ejecuta varias veces esta celda tras enviar el tercer mensaje, hasta ver que el DataFrame ha cambiado
agregadosDF.show()
retraso_medio_GRX_segundo_mensaje = 2.35
retraso_medio_MAD_segundo_mensaje = 12.7


+----+-------------+
|dest|retraso_medio|
+----+-------------+
+----+-------------+



In [ ]:
# Ejecuta varias veces esta celda tras enviar el cuarto mensaje, hasta ver que el DataFrame ha cambiado
agregadosDF.show()
retraso_medio_GRX_cuarto_mensaje = 2.5
retraso_medio_MAD_cuarto_mensaje = 12.7



+----+-------------+
|dest|retraso_medio|
+----+-------------+
+----+-------------+

